# One-Way ANOVA
<a id = "top"></a>
A one-way ANOVA allows us to compare the mean of a numerical variable between groups of a categorical variable.  The ANOVA test doesn't tell us *which* mean(s) are different from the others, just that at least one is significantly different.  We the follow that up with post-hoc multiple t-tests to determine which pairs are significantly different from each other.  These post-hoc tests adjust for the *multiple comparison problem*, which is inflation of Type I error due to repeatedly conducting hypothesis tests on the same data.

- [One-way ANOVA basics](#basics)
    - [Graphically looking at differences in group means](#graph)
    - [Decomposing Variance - Between vs. Within](#decomp)
    - [Conducting the F-test (ANOVA)](#ftest)
    - [Interpreting the Result](#interp)
- [Assumptions](#assump)
    - [Basic Assumptions](#baseassump)
    - [Homogeneity of Variances (Levene's Test)](#homovar)
    - [Normality of Residuals (QQ Plots)](#resid)
- [Post-hoc Pairwise Comparisons](#posthoc)
    - [Tukey's Honest Significant Difference (HSD)](#tukey)
    - [Bonferroni Adjustment](#bonf)
- [Effect Size](#effect)
    - [R-squared](#rsq)
    - [Cohen's $f^2$](#cohen)

In [ ]:
library(tidyverse)
library(magrittr)
library(ggpubr) # containes line/dot plot for visualizing means
library(DescTools) # contains levene's test function
library(pwr) # for power analysis
library(scales) ## for scaling functions for ggplot2

options(repr.plot.width=7, repr.plot.height=5) ## set options for plot size within the notebook -
# this is only for jupyter notebooks, you can disregard this.

<a id = "basics"></a>
## One-way ANOVA Basics
For this in-class example we're going to use a small example dataset built in to R - "warpbreaks."  
https://stat.ethz.ch/R-manual/R-patched/library/datasets/html/warpbreaks.html

We have three variables that describe weaving on 54 looms - 9 looms in each combo of wool and tension:
- breaks	numeric	The number of breaks
- wool	    factor	The type of wool (A or B)
- tension	factor	The level of tension (L, M, H)

We will see if the type of tension (low, medium, or high) has an effect on the number of breaks.  So we're looking to see if the mean number of breaks differs significantly by loom tension.

In [ ]:
# basic summary statistics
data(warpbreaks)
summary(warpbreaks)

[Return to Top](#top)
<a id = "graph"></a>
### Graphically looking at differences in group means.
We should always begin by visualizing the distribution of warp breaks within the three groups (low, medium, and high tension).  There are a number of ways that we can look at the difference in distributions.  Here I'm going to look at two - density curve and boxplot.

In [ ]:
## density plot
warpbreaks %>%
  ggplot( aes(x=breaks, fill=tension)) +
    geom_density(alpha=0.6) +
    scale_fill_manual(values=c("#26d5b8", "#ff5733", "magenta")) +
    labs(fill= "Tension",
         y = "Density",
         x = "Warp Breaks",
         title = "Distribution of warp breaks by loom tension")

In [ ]:
#boxplot
boxp <- warpbreaks %>%
  ggplot( aes(y=breaks, x=tension, fill=tension)) +
    geom_boxplot() +
    stat_summary(fun.y = mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y.., color = "mean"),
                 width = 0.75, linetype = "solid", size = 2) +
    stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), 
                geom="errorbar", color="#39ff14", width=0.2, size = 1) +
    scale_fill_manual(values=c("#26d5b8", "#ff5733", "magenta")) +
    scale_color_manual(values = "#39ff14")+
    labs(fill= "Tension",
         y = "Warp Breaks",
         x = "Loom Tension",
         color = "Group Mean",
         title = "Distribution of warp breaks by loom tension")
boxp

These both show us the distribution of the data, but they have their unique strengths.

The density plot allows us to visualize the shape of the distribution and assess the normality of the distributions.

The boxplot makes it easier to look at the mean and median of the distribution as well as compare the range of the distribution (IQR) which can tell us a bit about possible differences in within group variance.  Here, the IQR of H is very small, while the IQR of L is much larger, maybe 2x the range of H.  This can begin to show us that the within group variance likely differs.  We can also infer some things about the distribution here - the difference between the mean and the median in group L is substantial which indicates there's skewness in that distribution.  However, the exact shape of the distribution is more easily viewed in the density plot - where we can see the evidence of skewness that we detect in the boxplot.

There are other versions of plots we can use - you saw a variety plots in Lab 5 that allow you to plot the distribution of a numerical variable within groups defined by a categorical variable.  A histogram is roughly equivalent to a density plot - in terms of viewing the shape of the distribution.  A grouped mean plot shows us the mean and spread of the individual observations, similar to the box plot.

[Return to Top](#top)
<a id = "decomp"></a>
### Decomposing Variance - Between vs. Within
The first step in conducting an ANOVA analysis is calculating the sum of squares.  We decompose the total sum of squares into two pieces:
- SSB = the "between" group variance - the difference between the overall mean and the group means
- SSW = the "within" group variance (residuals) - the difference between the group mean and the individual observations.


We calculate the squared differences for each group, or each respondent, then sum these to get the overall sum of squares.  This is the first piece we need to begin to build our ANOVA table.  

For this example we're going to "manually" calculate these values, but you won't have to do this by hand when you do your own analyses.  Will make use of group_by(), summarize(), and mutate() to easily calculate these values.

First we'll calculate the group means and the deviations between the group means and the overall mean.

## $ SS_{between} = \sum_{i=1}^k N_k(\bar{x_i} - \bar{x})^2 $

In [ ]:
# create a df with group means and sample size in group (we'll need it later)
group_data <- warpbreaks %>% 
                group_by(tension) %>% 
                summarize(N_k = n(),
                          group_mean = mean(breaks))
group_data

In [ ]:
# add difference between group mean and overall mean as a new column
group_data %<>% mutate(diff = group_mean - mean(warpbreaks$breaks))
group_data

In [ ]:
# square the differences
group_data %<>% mutate(sq_diff = diff^2)
group_data

In [ ]:
# multiply the squared difference by the sample size of the group (n_k)
group_data %<>% mutate(sq_diff_times_n_k = sq_diff * N_k)
group_data

The overall sum of squares for the group (tension) is one value - the sum of the squared differences times $N_k$.  So we'll sum that last column to get our SSB value.

## $ SS_{between} = \sum_{i=1}^k N_k(\bar{x_i} - \bar{x})^2 $

In [ ]:
# sum all squared differences times group sample size to get overall SSB
SSB <- sum(group_data$sq_diff_times_n_k)
SSB

We now know our SSB is 2034.  We now need to calculate the variance within groups - SSW.  This the deviation between the individual observations and the group mean (of the group the observation belongs to).  

## $ SS_{within} = \sum_{i=1}^k \sum_{j=1}^{n_k}(x_{ij} - \bar{x_i})^2 $

In [ ]:
# calculate residual - diff between x_ij and group mean
within_group <- warpbreaks %>% 
                    group_by(tension) %>% 
                    mutate(diff = breaks - mean(breaks)) %>% 
                    # breaks is individual obs, mean(breaks) is group mean because of the group_by
                    ungroup() # ungroup because we're working with individual observations
within_group

In [ ]:
#square the differences
within_group %<>% mutate(sqdiff = diff^2)
within_group

The overall sum of squares for the residuals (within group)  is one value - the sum of the squared differences between observations and group means.  So we'll sum that last column to get our SSW value.

## $ SS_{within} = \sum_{i=1}^k \sum_{j=1}^{n_k}(x_{ij} - \bar{x_i})^2 $

In [ ]:
SSW <- sum(within_group$sqdiff)
SSW

[Return to Top](#top)
<a id = "ftest"></a>

### Conducting the F-test (ANOVA)
Recall that an ANOVA table looks something like this: 
![](anovatab.PNG)

Let's setup a df to begin putting these pieces together:

In [ ]:
breaks_aov <- data.frame(Source = c("Between Groups (tension)", "Within Groups (residual)"))
breaks_aov$SS <- c(SSB, SSW)
breaks_aov

To calculate our F-ratio we need the mean squares.  To calculate the mean squares we'll need to degrees of freedom.  So let's add those to our anova table.

In [ ]:
# Between group dof is number of groups minus 1
df_b <- nrow(group_data) - 1
# Within group dof is number of observations minus number of groups
df_w <- nrow(warpbreaks) - nrow(group_data)

# create a new column in the anova df
breaks_aov$df <- c(df_b, df_w)
breaks_aov

Now that we have those two pieces, we can easily calculate the mean squares => The sum of squares over degrees of freedom.

In [ ]:
# calculate MSB and MSW
MSB <- SSB / df_b
MSW <- SSW / df_w

# add to anova df
breaks_aov$MS <- c(MSB, MSW)
breaks_aov

Now we have everything we need to caluclate our F-ratio (F-value, F-score, F-statistic, all of these terms are interchangeable).

The F-ratio is the ratio of the between group variance to the within group variance.  As the between group variance gets larger and larger (in comparison to the Within group variance), the F-ratio gets larger and therefore is more likely to exceed the critical F-value.

In [ ]:
# Calculate F
F_ratio <- MSB / MSW

# add to anova df - because there's only one F value, the convention is to place it on the line of the group.  
# I will fill the second row with an NA
breaks_aov$F_ratio <- c(F_ratio, NA)
breaks_aov

[Return to Top](#top)
<a id = "interp"></a>

### Interpreting the Result

Finally, we need to determine our p-value.  We get the p-value from the F distribution - it's the probability under the curve from our F-value in the right tail.

Remember an F-value has two degrees of freedom - a numerator and denominator degrees of freedom.  We already have these values in our anova df.

In [ ]:
# the pf function gives us a probability under the F-distribution curve cut off at a certain f-value
# pf(fstat, df1, df2, lower.tail = FALSE)
pf(F_ratio, df_b, df_w, lower.tail = FALSE)

So there's the last piece - our p-value is 0.0017, which is below an alpha of 0.05.  Therefore we reject the null hypothesis.  Tension is a significant predictor of warp breaks.  The mean of breaks is significantly different in at least one group (but remember we don't know which group(s) are significantly different and in which directions from this value.  We will later look at pair-wise comparisons.

Let's check our results against running the `aov` function with the same data.

In [ ]:
summary(aov(breaks ~ tension, data = warpbreaks))

Our "hand calculated" values are identical to the ones output from the aov() function.  So we did everything correctly! 

 [Return to Top](#top)
<a id = "assump"></a>

## Assumptions
As with any statistical test, we have a number of assumptions that we have to fulfill in order for the test to be fully valid.  Let's review the ANOVA assumptions.

<a id = "baseassump"></a>
The basic ones we don't need to test:
1. Dependent variable is numeric - **breaks is numeric, so we're good here!**
2. Group sample sizes are approximately equal - **in this case they're all exactly the same size**
3. Independence of observations - **each loom is randomly assigned to wool/tension combos**
4. No extreme outliers - **we can review the boxplot - nothing seems too extreme**

<a id = "homovar"></a>
Pre-check item:
5. Homogeneity of variance - the within group variance for each of the groups should be equal.  This is the same assumption we had for our t-test, but in that case we only had two groups, now we have more.  **We need to test this using Levene's Test.  This is typically reviewed PRIOR to conducting your ANOVA analysis.**

Levene's Test is the same as var.test() we used with t-tests, however it tests the variances of multiple groups.

Recall:

$H_0:$ The variances in the groups are equal. <BR>
$H_A:$ The variances in the groups are not equal.

In this test, we sort of want to fail to reject null, because it's easier if our variances are equal and we don't need to make the adjustment.

In [ ]:
#LeveneTest(DV ~ IV, data = your data frame)

LeveneTest(breaks ~ tension, data = warpbreaks)

Again, similar to var.test() we get an F-value and a p-value.  Because our p-value is greater than an alpha of 0.05, we fail to reject null.  This means we can conclude that the variances are equal (or equal enough to not violate the assumption).

**IMPORTANT**: This is a test of variance, but this is **NOT** the ANOVA test.  This just tests the assumption of homogeneity of variances.  We cannot use these results to make inference about our means.

<a id = "resid"></a>

So, we have one final assumption:

6. Normality of _**residuals**_

Here we don't care about the normality of our observations (breaks) we care about the normality of the residuals (the difference between each observation of breaks and the group mean for that observation).  These residuals should be normally distributed.  Because the residuals are calculated as a result of conducting the analysis, we cannot check this assumption until after we conduct our ANOVA test.  So this is considered a "post-hoc" check of assumptions.

**We check this via a QQ plot of the _residuals_ from our data.**

In [ ]:
## we can get the residuals from an aov object (the results of running aov)
tension_aov <- aov(breaks ~ tension, data = warpbreaks)
resid_df <- data.frame(resid = tension_aov$residuals) ## the residuals part of the aov results using $residuals

resid_df %>% ggplot(aes(sample = resid)) +
  geom_qq_line(color = "red", size = 1) +
  geom_qq(color = "black") +
  labs(title = "QQ Plot of Residuals")

Our residuals seem to be relatively normal (they closely follow the reference line) with a slight amount of deviation in the right tail (two possible outliers?).

[Return to Top](#top)
<a id = "posthoc"></a>

## Post-hoc Pairwise Comparisons
When we have a significant result from an ANOVA analysis, our natural next question is - which groups are significantly different from each other?  We can use a series of pairwise comparisons after (post-hoc) our ANOVA analysis to investigate this.

Because repeatedly t-testing the same data can inflate Type I error (alpha) we need to make adjustments to account for that.  There are two methods - Tukey's Honest Significant Difference (HSD) or the Bonferroni adjustment.

<a id = "tukey"></a>
### Tukey's Honest Significant Difference (HSD)
This procedure is an adjustment to the typical t-test procedure to account for the multiple comparisons issue.  It uses a $q$ distribution for critical values.  In the denominator of the test the standard error is calculated with MSW and group sample sizes instead of standard deviation and sample sizes.
https://www.statisticshowto.datasciencecentral.com/tukey-test-honest-significant-difference/

I won't show the "by hand" calculation, but let's review the output for this example.

In [ ]:
#use the TukeyHSD function and pass it your saved aov object.

TukeyHSD(tension_aov) #aov object created in section QQ plot section

We get a difference, lower and upper CI values for that difference, and an "adjusted" p-value (correction for multiple comparisons).  It will give us every possible pair of the groups of our categorical variables. 

From this we can review the p-values and see that there is a significant difference in the average number of breaks with medium vs. low tension, and also with high vs. low tension.  However, the difference between the mean breaks for high vs. medium tension is not significant.  So low tension is significantly different from both high and medium groups, but high and medium and not significantly different from each other.

Let's remind ourselves of the distributions of breaks within these groups to visualize the differences reflected in these numbers.

In [ ]:
boxp

If you're wondering about the total number of possible pairwise combos with your factor levels, this is a combination calculation.

## ${n \choose r} = \frac{n!}{r!(n-r)!} $

In our case - 
## ${3 \choose 2} = \frac{3!}{2!(3-2)!} = \frac{3\times2\times1}{(2\times1)(1)} = \frac{6}{2} = 3$

But it's not important to actually figure that out - you can just run the code in R and view all of the pairs.

[Return to Top](#top)
<a id = "bonf"></a>
### Bonferroni Adjustment
The Bonferroni adjustment (or Bonferroni correction) is not a procedure specific to pairwise t-tests.  What it does is adjust the p-values to reduce the inflation of Type I error due to the multiple comparisons.  The Bonferroni adjustment is considered to be more conservative than Tukey HSD (yields higher p-values), however, if you only want to test some, but not all, of the pairwise comparisons it might be more appropriate than Tukey HSD, because alpha is split only across the paired tests you conduct.

Here we're going to use the defaults, and get all possible pairs.

In [ ]:
# for bonferroni we use the function pairwise.t.test with the p.adj argument set to "bonf"
pairwise.t.test(warpbreaks$breaks, warpbreaks$tension, p.adj = "bonf")

In this output we get just the p-values, and they are displayed in a table similar to a correlation matrix, where the row name and col name indicate what the pair comprised of.  Same conclusion as above with Tukey HSD for significant pairs, however you can note that these p-values are in fact a bit higher than those from the Tukey HSD.

[Return to Top](#top)
<a id = "effect"></a>

## Effect Size
A statistical analysis is not complete without addressing the substantive significance of a result.  Just because we find that our analysis is statistical significant doesn't mean that it is large enough to matter. For ANOVA and throughout the end of the semester we will be focusing on measures of effect size that tell us the strength of our IV in predicting our DV.  In other words, how much of the variance in our outcome is explained by our predictor variable.

In the case of ANOVA - how much of the variance in the numerical variable is explained by group/level of the categorical variable.  Is the main source of variation in the scores due to the different group means, or due to individual variations in scores (residuals)?

<a id = "rsq"></a>
### R-squared
Our effect size for ANOVA is the same r-squared statistic we looked at in two-sample t-tests.  The interpretation is the same, the formula for calculating it is different.  In the ANOVA situation r-squared is sometimes referred to as eta-squared, but it is the same thing.

## $r^2 = \frac{SS_{between}}{SS_{total}}$

It calculates the proportion of the total variation that is explained by the groups (between-group SS).

In our case we haven't calculated SST, so we can use SSB + SSW in the denominator.

**IMPORTANT**: You need to use SSB and SST, not the mean square (MSB).

In [ ]:
# calculate r-squared
# we calculated SSB and SSW in the decomposing variance section.
rsq = SSB / (SSB + SSW)
rsq # proportion
percent(rsq, accuracy = .01) # percentage

So the r-squared for the warpbreaks data shows that 22% of the variance in breaks can be explained by the level of tension used on the loom. 

The "rule of thumb" guide for r-squared is:
$r^2$ ranges from 0 to 1 where 0 means there is no variation explained by the IV and 1 means all of the variation is explained by the IV.
- $r^2 \approx$ 0.1, little to no effect
- $r^2 \approx$ 0.3, weak effect
- $r^2 \approx$ 0.5, moderate effect
- $r^2 \approx$ 0.6 to 1, strong effect

However, IMHO, 22% is a substantial amount of variance explained by one variable, and is actually pretty good.  In my opinion these rule of thumb guidelines are a bit impractical for the real world experience of data analysis, where an r-squared value of 0.4 (40%) might be seen as particularly good.  

[Return to Top](#top)
<a id = "cohen"></a>

### Cohen's $f$
The other effect size statistic we will use is Cohen's $f$.  Cohen's $f$ is primarily needed because it is the effect size used in power calculations.  R-squared is preferred for "interpretation" purposes.  Cohen's $f$ is calculated using the r-squared value.

## $f = \sqrt{\frac{r^2}{1 - r^2}}$

In [ ]:
## calculate cohen's f using saved value of rsq
cohenf <- sqrt(rsq / (1-rsq))
cohenf

Cohen's f can be interpreted similarly to Cohen's d, but as a standardized average effect size across all the levels of the categorical variable.  But we will use r-squared moving forward in the class, so become familiar with it now.  It also has a more straightforward interpretation once you can wrap your head around the understanding of "proportion of variance explained."

But, Cohen's f will be needed for the power calculation.  You cannot use r-squared in the power function (which you will see in lab).